<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_3_tuning_healthnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    966      0 --:--:-- --:--:-- --:--:--   969
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-3.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [51]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-3.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthritis
4,8,arthritis


In [52]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-3.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,5,rapid heart beat
2,6,diastolic dysfunction
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_healthnews_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [57]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [61]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.965698,5.823224,0.141071,02:13


In [62]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.777234,5.257009,0.187165,02:12


In [63]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.401062,5.210304,0.193583,02:13


In [64]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.343609,5.164712,0.199498,02:12


In [0]:
learn.save_encoder('healthnews_fold3_first')

In [66]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('healthnews_fold3_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.698810,4.326478,0.311705,01:10


In [67]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.906377,4.025083,0.335878,01:31


In [68]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.624346,3.798762,0.361323,03:40


In [69]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.446486,3.607296,0.398219,03:27
1,3.248147,3.523982,0.403308,03:43


In [70]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.133181,3.401990,0.428753,03:35
1,2.966935,3.375318,0.430025,03:36


In [71]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.915335,3.335425,0.440204,03:46
1,2.776113,3.228817,0.454198,03:42


In [72]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.747694,3.221101,0.451654,03:45
1,2.669546,3.166880,0.454198,03:43


In [73]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.569214,3.128708,0.461832,03:37
1,2.588912,3.187542,0.461832,03:33


In [74]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.502688,3.198145,0.468193,03:32
1,2.561022,3.230850,0.436387,03:34
2,2.434128,3.063876,0.477099,03:29
3,2.355459,3.063705,0.479644,03:48


In [75]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.381025,3.116209,0.487277,03:28
1,2.411828,3.109181,0.441476,03:36
2,2.381588,3.140239,0.488550,03:26
3,2.262901,3.104896,0.486005,03:24


In [76]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.298053,3.127789,0.451654,03:30
1,2.386601,3.182104,0.452926,03:28
2,2.233283,3.114327,0.494911,03:34
3,2.362511,3.243953,0.491094,03:28


In [77]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.262852,3.122320,0.492366,03:32
1,2.322556,3.302573,0.482188,03:24
2,2.209961,3.134391,0.488550,03:44
3,2.097444,3.120465,0.491094,03:49


In [78]:

learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.135046,3.146594,0.484733,03:54
1,2.229097,3.180576,0.484733,03:53
2,2.194897,3.169557,0.492366,03:42
3,2.073400,3.122192,0.493639,03:28


In [79]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.142970,3.214613,0.487277,03:46
1,2.320973,3.391159,0.460560,03:37
2,2.219083,3.231718,0.492366,03:33
3,2.080455,3.158566,0.494911,03:31


In [80]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.056024,3.172817,0.487277,03:47
1,2.175601,3.299092,0.487277,03:25
2,2.097759,3.230633,0.478372,03:30
3,2.043834,3.205131,0.487277,03:52


In [81]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.090266,3.286829,0.489822,03:36
1,2.113033,3.224156,0.486005,03:41
2,2.055064,3.236005,0.488550,03:30
3,1.997898,3.234527,0.489822,03:28


In [82]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.089975,3.331400,0.486005,03:40
1,2.139725,3.280981,0.487277,03:43
2,2.122406,3.357355,0.493639,03:40
3,2.098382,3.341384,0.493639,03:37


In [83]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.081818,3.326484,0.487277,03:36
1,2.131535,3.283311,0.487277,03:29
2,2.047410,3.269719,0.487277,03:38
3,1.985931,3.297161,0.487277,03:33


In [84]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.068855,3.388449,0.484733,03:44
1,2.119370,3.322744,0.483461,03:34
2,2.174737,3.326721,0.488550,03:36
3,2.130074,3.410383,0.484733,03:45


In [0]:
# save the best model

learn.save_encoder('ask-3-tuning-healthnews')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [87]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 5, 1035, 1026, 1026, 1026, 1026, 933, 1035, 27, 27, 690, 1035, 30, 584, 1035, 1035, 1028, 37, 37, 37, 37, 37, 37, 896, 301, 896, 328, 1035, 49, 1028, 49, 421, 49, 49, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 1002, 61, 61, 4, 1028, 100, 276, 686, 1035, 72, 746, 1035, 209, 289, 864, 164, 741, 741, 741, 1028, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 289, 1028, 1028, 289, 80, 80, 80, 89, 1028, 214, 214, 328, 1026, 1035, 1035, 826, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 289, 427, 96, 1028, 334, 217, 164, 214, 408, 214, 214, 861, 1028, 55, 180, 108, 717, 108, 864, 111, 289, 189, 117, 896, 1028, 145, 1028, 1028, 125, 1028, 125, 125, 125, 125, 125, 896, 289, 79, 753, 704, 622, 473, 473, 473, 140, 226, 447, 164, 809, 577, 517, 3, 1028, 1028, 167, 289, 55, 604, 111, 111, 462, 686, 686, 427, 1028, 897, 182, 182, 289, 289, 183, 183, 183, 183, 758, 758, 758, 758, 758, 758, 240, 240, 240, 

In [88]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

215
0.24826789838337182
